In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-16:] == 'image-to-tex-OCR':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/PROJECT_AMINE/image-to-tex-OCR/Jupyter_Notebooks
Current path:/Users/gregory/PROJECT_ML/PROJECT_AMINE/image-to-tex-OCR


In [2]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
from Data.Data_Module_CFG import Data_Module_CFG
from Models.Printed_Tex_Transformer import ResNetTransformer
from Lightning_Models.Printed_Tex_Lit_Model import LitResNetTransformer
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from PIL import Image
import torch
from pytorch_lightning.loggers import TensorBoardLogger
from torchvision import transforms
from PIL import Image
from IPython.display import display, Math
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps")
torch.cuda.is_available()

False

In [4]:


class CFG:
    
    device = dev
    
   
    use_wandb = True
    
    ######### Dataset #########
    
    # stage: 'train' or 'test'
    stage = 'train'
    set_max_label_length = 128
    number_png_images_to_use_in_dataset=270
    labels_transform='default'
    image_transform_name ='alb'
    train_val_fraction = 0.999
    vocabulary_path = 'Tokenizer/230k_ver2.json'
    load_vocabulary = True
    image_padding = True
    max_width = 1000


    batch_size = 8
    num_workers = 0
    on_gpu = False
    
    ######## Model and Lightning Model paramters ############
    
    # MODEL
    
    
    
    
    
    checkpoint = None
    save_directory = None
    
    
    accumulate_grad_batches = 128 // batch_size  # experiments showed batch_size * accumulate_grad = 192 is optimal
    learning_rate =  1e-5
    eta_min = 1e-8
    t_max = 40
    max_epochs = 35
    weight_decay =  1e-5
    precision =16
    
    # checkpointing
    save_top_k=5
    
    monitor="FBETA"
    mode="max"
    
    
    ####### Augemtnations ###############
    
    # Training Aug
    train_transforms = [
        # A.RandomResizedCrop(
        #     size, size, scale=(0.85, 1.0)),
        #A.Resize(patch_size, patch_size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.75),
        A.ShiftScaleRotate(p=0.75),
        
        A.RandomRotate90(p=0.5),
        A.Transpose(p=0.25),
        A.augmentations.geometric.transforms.Affine(scale=None, 
                                                    translate_percent=(0.05, 0.2),
                                                    translate_px=None, 
                                                    rotate=(-1, 1),
                                                    shear=None,
                                                    interpolation=1,
                                                    #mask_interpolation=0,
                                                    cval=0, 
                                                    cval_mask=0,
                                                    p=0.25) ,
        
        A.augmentations.geometric.transforms.ShiftScaleRotate(shift_limit=0,
                                                             scale_limit=(-.2, 0.1), 
                                                            rotate_limit=1.5,
                                                           border_mode=0,
                                                           p=0.25),
        
        A.OneOf([
                A.GaussNoise(var_limit=[10, 50]),
                A.GaussianBlur(),
                A.MotionBlur(),
              A.augmentations.geometric.transforms.OpticalDistortion(distort_limit=0.1,
                                                                  shift_limit=0.02,
                                                                  interpolation=1,
                                                                  border_mode=cv2.BORDER_CONSTANT,
                                                                   value=0,
                                                                  mask_value=0,
                                                                  always_apply=False,
                                                                   ),
           
             
                ], p=0.4),
        
       
        A.OneOf([  A.augmentations.geometric.transforms.ElasticTransform(alpha=120,
                                                                  sigma=120*0.05,
                                                                  alpha_affine=120 * 0.03,
                                                                  interpolation=1,
                                                                  border_mode=cv2.BORDER_CONSTANT,
                                                                  value=0,
                                                                  mask_value=0,
                                                                  always_apply=False,
                                                                  approximate=False,
                                                                  same_dxdy=False,
                                                                  #p=0.25
                                                                        ),
            A.GridDistortion(num_steps=5, distort_limit=0.3, ),
                 
             ], p=0.4),
        
        
       
        
        # A.Cutout(max_h_size=int(size * 0.6),
        #          max_w_size=int(size * 0.6), num_holes=1, p=1.0),
     
        ToTensorV2(transpose_mask=True),
    ]
    

    
    # Validaiton Aug
    val_transforms = [
        #A.Resize(patch_size, patch_size),
       
        ToTensorV2(transpose_mask=True),
    ]
    
    # Test Aug
    test_transforms = [
        #A.Resize(patch_size, patch_size),
      

        ToTensorV2(transpose_mask=True),
    ]
        
    
    

In [6]:
# Generate Data Module by uploading images and formulas
# images need to be in the folder Data/Data_Bank/generated_png_images
# formulas need to be in Data/Data_Bank/final_png_formulas.txt
# image filenames need to be in Data/Data_Bank/corresponding_png_images.txt

dataset = Data_Module_CFG(cfg=CFG
                )


# Generates DataFrame and vocabulary, tokezniers etc
#dataset.prepare_data()

# prepeares the dataloaders with transformations and splits train/val
dataset.setup(stage = 'train')

AttributeError: 'Data_Module_CFG' object has no attribute 'stage'